In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
12825,2022-07-04 19:55:31,2022-07-04,19:55:31,19,13.67187,0.03592,0.491094,0.000491
12826,2022-07-04 19:56:32,2022-07-04,19:56:32,19,13.74512,0.02549,0.350363,0.000350
12827,2022-07-04 19:57:32,2022-07-04,19:57:32,19,13.64746,0.02585,0.352787,0.000353
12828,2022-07-04 19:58:32,2022-07-04,19:58:32,19,13.67187,0.01682,0.229961,0.000230
12829,2022-07-04 19:59:32,2022-07-04,19:59:32,19,13.62305,0.00335,0.045637,0.000046


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000', '2022-06-27T00:00:00.000000000',
       '2022-06-28T00:00:00.000000000', '2022-06-29T00:00:00.000000000',
       '2022-06-30T00:00:00.000000000', '2022-07-01T00:00:00.000000000',
       '2022-07-02T00:00:00.000000000', '2022-07-03T00:00:00.000000000',
       '2022-07-04T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

20

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
11628,2022-07-03,23,0.0
11629,2022-07-03,23,0.0
11630,2022-07-03,23,0.0
11631,2022-07-03,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
...,...,...,...
211,2022-07-03,19,0.596466
212,2022-07-03,20,0.011871
213,2022-07-03,21,0.000000
214,2022-07-03,22,0.000000


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
data_selection = {'Mostly sunny': 3,
                  'Partly sunny': 2,
                  'Partly cloudy': 2,
                  'Intermittent clouds': 2,
                  'Partly sunny w/ showers': 1.5,
                  'Partly cloudy w/ showers': 1.5,
                  'Mostly clear': 1,
                  'Clear': 1,
                  'Mostly cloudy w/ t-storms': 0.5,
                  'Mostly cloudy': 0.5,
                  'Showers': 0.5,
                  'Cloudy': 0.5,
                  'Thunderstorms': 0.5,
                  'Mostly cloudy w/ showers': 0.5,
                  'Rain': 0}

In [11]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data['modified_weather_status'] = weather_data['weather status'].map(data_selection)

# weather_data.loc[(weather_data['CloudCover (%)'] >= 91) & (weather_data['CloudCover (%)'] <= 100), 'CloudCover (%)'] = 1
# weather_data.loc[(weather_data['CloudCover (%)'] >= 81) & (weather_data['CloudCover (%)'] <= 90), 'CloudCover (%)'] = 2
# weather_data.loc[(weather_data['CloudCover (%)'] >= 71) & (weather_data['CloudCover (%)'] <= 80), 'CloudCover (%)'] = 3
# weather_data.loc[(weather_data['CloudCover (%)'] >= 61) & (weather_data['CloudCover (%)'] <= 70), 'CloudCover (%)'] = 4
# weather_data.loc[(weather_data['CloudCover (%)'] >= 51) & (weather_data['CloudCover (%)'] <= 60), 'CloudCover (%)'] = 5
# weather_data.loc[(weather_data['CloudCover (%)'] >= 41) & (weather_data['CloudCover (%)'] <= 50), 'CloudCover (%)'] = 6
# weather_data.loc[(weather_data['CloudCover (%)'] >= 31) & (weather_data['CloudCover (%)'] <= 40), 'CloudCover (%)'] = 7
# weather_data.loc[(weather_data['CloudCover (%)'] >= 21) & (weather_data['CloudCover (%)'] <= 30), 'CloudCover (%)'] = 8
# weather_data.loc[(weather_data['CloudCover (%)'] >= 11) & (weather_data['CloudCover (%)'] <= 20), 'CloudCover (%)'] = 9
# weather_data.loc[(weather_data['CloudCover (%)'] >= 0) & (weather_data['CloudCover (%)'] <= 10), 'CloudCover (%)'] = 10

# weather_data['code'] = pd.factorize(weather_data['weather status'])[0]
weather_data.loc[weather_data['SolarIrradiance (W/m2)'] == 0, ['modified_weather_status', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
weather_data.drop(['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)'], axis = 1, inplace = True)
# weather_data['weather status'].unique()
# weather_data.tail(14)
# ce = weather_data[weather_data['code'] == 0 ][['code', 'weather status']]
# weather_data['CloudCover (%)'].unique()

In [12]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
# df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),Temp (°C),Hum (%),CloudCover (%),modified_weather_status
215,0.0,0.0,0,0,0.0
216,NaN,0.0,0,0,0.0
217,NaN,0.0,0,0,0.0
218,NaN,0.0,0,0,0.0
219,NaN,0.0,0,0,0.0
220,NaN,0.0,0,0,0.0
221,NaN,10.5,93,46,2.0
222,NaN,11.3,93,46,2.0
223,NaN,12.6,88,46,2.0
224,NaN,14.1,79,40,2.0


In [13]:
count_total_rows = len(df1) - 24

In [14]:
independent_columns = df1[['Temp (°C)', 'Hum (%)', 'CloudCover (%)', 'modified_weather_status']][0:count_total_rows]

In [15]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [16]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

#### Is model well fit or not?

In [17]:
r_sq = reg.score(independent_columns, dependent_column)
r_sq

0.40200237832614116

In [18]:
reg.intercept_

0.0

In [19]:
reg.coef_

array([ 0.08490217, -0.00562615, -0.00095514, -0.10153394])

In [20]:
forcasted_data = df1[['Temp (°C)', 'Hum (%)', 'CloudCover (%)', 'modified_weather_status']].tail(24)

In [21]:
return_array = list(reg.predict(forcasted_data))

In [22]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [23]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [24]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [25]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [26]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-07-04,0,0.000000
1,2022-07-04,1,0.000000
2,2022-07-04,2,0.000000
3,2022-07-04,3,0.000000
4,2022-07-04,4,0.000000
5,2022-07-04,5,0.121236
6,2022-07-04,6,0.189158
7,2022-07-04,7,0.327661
8,2022-07-04,8,0.511381
9,2022-07-04,9,0.681730


In [27]:
data_dataframe['Power (KW)'].sum()

13.960926461242018

## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [28]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], 
                                           data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.18488419476451834

In [29]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.42998162142644925

In [30]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], 
                                            data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.29151085231260077

#### R squared

In [31]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], 
                             data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.6286311017588672

#### Yesterday results

In [32]:
filter_last_day_values = df[df['Date'] == unique_date[-2]][['Date', 'Hour', 'Power (KW)']]
last_day_hourly_values = filter_last_day_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

filter_yes_values = df[(df['Date'] >= '2022-06-25') & (df['Date'] <= unique_date[-3])][['Date', 'Hour', 'Power (KW)']]
yes_hourly_values = filter_yes_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data1 = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')

weather_data1['modified_weather_status'] = weather_data1['weather status'].map(data_selection)
weather_data1.loc[weather_data1['SolarIrradiance (W/m2)'] == 0, ['modified_weather_status', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
weather_unique_date = weather_data1['Date'].unique()
filter_weather_yes_values = weather_data1[
                                         (weather_data1['Date'] >= '2022-06-25') & 
                                         (weather_data1['Date'] <= weather_unique_date[-3])][['Temp (°C)', 'Hum (%)', 'CloudCover (%)', 'modified_weather_status']]
yes_df1 = pd.concat([yes_hourly_values, filter_weather_yes_values], axis = 1)
yes_df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
yes_count_total_rows = len(yes_df1)
yes_independent_columns = yes_df1[['Temp (°C)', 'Hum (%)', 'CloudCover (%)', 'modified_weather_status']][0:yes_count_total_rows]
yes_dependent_column = yes_df1['Power (KW)'][0:yes_count_total_rows]
rfr_yes = linear_model.LinearRegression(fit_intercept = False)
rfr_yes.fit(yes_independent_columns, yes_dependent_column)
forcasted_yes_values = weather_data1[(weather_data1['Date'] == weather_unique_date[-2])][['Temp (°C)', 'Hum (%)', 'CloudCover (%)', 'modified_weather_status']]
rfr_yes_return_array = rfr_yes.predict(forcasted_yes_values)
rfr_yes_predicted_data = pd.DataFrame(rfr_yes_return_array, columns = ['Power (KW)'])
rfr_yes_pe = rfr_yes_predicted_data['Power (KW)'].sum()
rfr_yes_mse = metrics.mean_squared_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rmse = np.sqrt(rfr_yes_mse)
rfr_yes_mae = metrics.mean_absolute_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rs = metrics.r2_score(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rmse

0.49680270495987844

In [33]:
# dependent_variable = 'Power (KW)'

In [34]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [35]:
# independent_variables

In [36]:
# X = df1[independent_variables].values

In [37]:
# y = df1[dependent_variable].values

In [38]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
# X_train

In [40]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [41]:
# y_pred = regressor.predict(X_test)
# y_pred

In [42]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [43]:
# r2_score(y_test, y_pred)